In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
import yfinance as yfin

%matplotlib inline

In [ ]:
yfin.pdr_override()

# Read Data
test = data.get_data_yahoo(['FB','TSLA'], start='2018-01-01', end='2019-12-31')
test.head()

In [ ]:
# Closing price
test = test['Adj Close']
test.head()

In [ ]:
### Log is time additive
from math import log as ln

p1 = 100
p2 = 110 
p3 = 120

print(ln(p2/p1)) #= ln(110/100) = 9.53%

print(ln(p3/p2))


print(ln(p2/p1) + ln(p3/p2))

print(ln(p3/p1))


In [ ]:
tesla = test['TSLA'].pct_change().apply(lambda x: ln(1+x))
tesla.head()

In [ ]:
# Variance
var_tesla = tesla.var()
var_tesla

In [ ]:
# Log of Percentage change for Facebook
fb = test['FB'].pct_change().apply(lambda x: np.log(1+x))
fb.head()

In [ ]:
# Variance
var_fb = fb.var()
var_fb

In [ ]:
# Volatility
tesla_vol = np.sqrt(var_tesla * 250)
fb_vol = np.sqrt(var_fb * 250)
tesla_vol, fb_vol

In [ ]:
test.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250)).plot(kind='bar')

In [ ]:
# Log of Percentage change
test1 = test.pct_change().apply(lambda x: np.log(1+x))
test1.head()

In [ ]:
# Covariance
test1['TSLA'].cov(test1['FB'])

In [ ]:
test1['TSLA'].corr(test1['FB'])

In [ ]:
test2 = test.pct_change().apply(lambda x: np.log(1+x))
test2.head()

In [ ]:
w = [0.2, 0.8]
e_r_ind = test2.mean()
e_r_ind

In [ ]:
# Total expected return
e_r = (e_r_ind*w).sum()
e_r

In [ ]:
# Import data
df = data.get_data_yahoo(['HBOR3.SA','FLRY3.SA','MGLU3.SA','WEGE3.SA'], start='2017-01-01', end='2021-10-20')
df.tail()

In [ ]:
# Closing price
df = df['Adj Close']
df.tail()

In [ ]:
# Log of percentage change
cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

In [ ]:
corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()
corr_matrix

In [ ]:
# Yearly returns for individual companies
ind_er = df.resample('Y').last().pct_change().mean()
ind_er

In [ ]:
# Portfolio returns
w = [0.4, 0.3, 0.2]
port_er = (w*ind_er).sum()
port_er

In [ ]:
# Volatility is given by the annual standard deviation. We multiply by 250 because there are 250 trading days/year.
ann_sd = df.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))
ann_sd

In [ ]:
assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']
assets

In [ ]:
p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(df.columns)
num_portfolios = 10000
for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

In [ ]:
data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]
portfolios  = pd.DataFrame(data)
portfolios.head() # Dataframe of the 10000 portfolios created

In [ ]:
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])

In [ ]:
min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()]
# idxmin() gives us the minimum value in the column specified.                               
min_vol_port

In [ ]:
# plotting the minimum volatility portfolio
plt.subplots(figsize=[10,10])
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)

In [ ]:
# Finding the optimal portfolio
rf = 0.01 # risk factor
optimal_risky_port = portfolios.iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
optimal_risky_port

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)


https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/#1-what-is-portfolio-optimization-